# Analysis of Treasury Bill Pricing and Interest Rate Risk

## Background
Treasury bills, otherwise known as T-Bills, are financial instruments with short-term due dates ranging from a few days to 52 weeks. They are fixed-income investments with zero coupons, meaning no coupon payments are made during the term. Instead, the price of treasury bills is calculated so that the bill receiver gets the face (par) value ($V_{P}$) at the end of the term. 

<div>
    <center>
        <img src="figs/Fig-ZC-Schematic.png" width="440"/>
    </center>
</div>

### Pricing
A zero-coupon T-bill with an annual effective market interest rate $\bar{r}$, which is specified at the time of purchase, and a term of T years has a _fair price_ of:

$$
V_{B} = \mathcal{D}_{T,0}^{-1}\cdot{V_{P}}
$$

where $\mathcal{D}_{T}$ denotes the discount factor from the time of the auction to the term of T-bill, i.e., $0\rightarrow{T}$. Typically, a constant averaged rate of interest is used to compute the discount factor:

$$
\mathcal{D}_{T,0} = (1+\bar{r})^{T}
$$

However, dynamic formulations of the discount factor with period-specific interest rates can also be used:

$$
\mathcal{D}_{T,0} = \left[\prod_{j=0}^{T-1}\left(1+r_{j+1,j}\right)\right]
$$

where $r_{j+1,j}$ terms are called short rates, i.e., the interest rate earned between period $j\rightarrow{j+1}$. Treasury bills are released for specific periods $T=$ 4, 8, 13, 26, and 52 weeks. The fair price $V_{B}$ of the T-bill is the future face value $V_{P}$ that is discounted to today’s value by the effective market interest rate $\bar{r}$, which is assumed constant over the lifetime of the treasury bill.

## Project
You purchased a 52-week T-bill with a par value $V_{P}=100.0$ USD and an effective market rate of interest $\bar{r}$ (annualized) on January 03, 2022. However, some time after purchasing the T-bill, the interest rate is now $\bar{r}^{\prime}$. Should you `sell` the T-bill before it matures or `hold` the T-bill to term?

### Objectives
 1. Load, and analyze historical T-bill interest rate data sets from 2019 to 2023 computed daily by [Federal Reserve Bank of New York](https://www.newyorkfed.org). Store these data in a [Dictionary](https://docs.julialang.org/en/v1/base/collections/#Dictionaries). Select the year`"2022"` for analysis. 
 1. Develop a strategy to decide between a `hold` or `sell` action for your T-bill: 
     * Test your strategy as you move forward in time from January 03, 2022 to the last trading day of the year.
     * Hypothetically, assume you purchased the T-bill on the last trading day of the year, and using a time machine, went backward in time until January 03, 2022. How does your strategy perform?

## Setup
In the following code block we setup the computational aspects of the problem by including the `Include.jl` file. The `Include.jl` file loads external packages, functions that we will use in this excercise and custom types to model the components of our problem.

### Packages
`Include.jl` loads several external packages that we will use for our excercise:
* [DataFrames.jl](https://dataframes.juliadata.org/stable/) provides a set of tools for working with tabular data in [Julia](https://julialang.org). Its design and functionality are similar to those of [Pandas (in Python)](https://pandas.pydata.org) and [data.frame, data.table and dplyr (in R)](https://dplyr.tidyverse.org), making it a great general purpose data science tool.
* [Plots.jl](https://docs.juliaplots.org/stable/) is a plotting library in [Julia](https://julialang.org).

### Functions
`Include.jl` loads the following [Julia functions](https://docs.julialang.org/en/v1/manual/functions/) into the notebook:

##### `loadratesfile(year::String = "2022") -> DataFrame`

> This function takes a [String](https://docs.julialang.org/en/v1/manual/strings/) encoding the year in the `YYYY` format, e.g., `"2019"` or `"2020"` and returns a [DataFrame](https://dataframes.juliadata.org/stable/) holding the daily interest rates for US Treasury Bills for various durations found on [Treasury.gov](https://home.treasury.gov/resource-center/data-chart-center/interest-rates/TextView?type=daily_treasury_bill_rates). This project has data from `"2019"`, `"2020"`, `"2021"`, `"2022"` and `"2022"`. The year parameter has a default value of `"2022"`.

##### `build(model::Type{MyUSTreasuryBillModel}, data::NamedTuple) -> MyUSTreasuryBillModel`

> This function takes information in the `data` [NamedTuple](https://docs.julialang.org/en/v1/base/base/#Core.NamedTuple) argument (the par value, the rate and the duration of the T-bill) and returns an instance of the `MyUSTreasuryBillModel` custom type.

##### `build(model::Type{MySymmetricBinaryLatticeModel}, data::NamedTuple)::MySymmetricBinaryLatticeModel` and `build(model::Type{MyBinaryLatticeNodeModel}, data::NamedTuple)::MyBinaryLatticeNodeModel` 

> These factory functions take information in the `data` [NamedTuple](https://docs.julialang.org/en/v1/base/base/#Core.NamedTuple) argument and construct a node or the binomial short-rate lattice model, respectively. 

##### `price(model::MyUSTreasuryBillModel, compounding::T) -> MyUSTreasuryBillModel where T <: AbstractCompoundingModel`

> This function takes a `MyUSTreasuryBillModel` instance, and a compounding model and returns an updated `MyUSTreasuryBillModel` instance that holds the price $V_{B}$ value in the `price` field of the `MyUSTreasuryBillModel` instance.

##### `𝔼(model::MySymmetricBinaryLatticeModel; level::Int = 0) -> Float64` and `𝕍(model::MySymmetricBinaryLatticeModel; level::Int = 0) -> Float64`

> These functions take a`MySymmetricBinaryLatticeModel` instance, and the level and the tree (`0`-based) and compute the [expectation](https://en.wikipedia.org/wiki/Expected_value) and [variance](https://en.wikipedia.org/wiki/Variance) of the short-rates for the given level (time-step) of the lattice. 

### Types
`Include.jl` loads some [problem-specific types](https://docs.julialang.org/en/v1/manual/functions/) that will be helpful for the analysis of T-bill pricing. 

* `MyUSTreasuryBillModel` is a [mutable type](https://docs.julialang.org/en/v1/manual/types/#Mutable-Composite-Types) holding the par value $V_{P}$, the duration $T$ and the interest rate $\bar{r}$ for a zero-coupon treasury bill. You construct a `MyUSTreasuryBillModel` instance using the `build` method described above.
* `DiscreteCompoundingModel` and `ContinuousCompoundingModel` are [immutable types](https://docs.julialang.org/en/v1/manual/types/#Composite-Types) that let our code know which compounding model we wish to use.
* The `MyBinaryLatticeNodeModel` and `MySymmetricBinaryLatticeModel` mutable types hold information about the nodes, and the binomial lattice, respectively, for the short-rate calculations. 

In [1]:
include("Include.jl");

  Activating project at `~/Desktop/julia_work/CHEME-130-eCornell-Repository/courses/CHEME-131/module-1`


## Set constant values 
We will need several constants, regardless of the year we choose to analyze. First, we will always analyze 52-week data, so we encode this choice in the `key` variable. Next, we'll assume a discrete compounding model; thus, we instantiate an instance of the `DiscreteCompoundingModel` type and then store this instance in the `compounding` variable. Lastly, the T-bill’s face (par) value will always be $V_{P}$ = 100.0 USD.

In [2]:
number_of_weeks_per_year = 52;
key = Symbol("52 WEEKS COUPON EQUIVALENT");
Vₚ = 100.0;
T = (52.0/number_of_weeks_per_year);
compounding = DiscreteCompoundingModel();

## Objective 1: Load and analyze historical interest rate ranges
First, we load the interest rate data for each year in our data set, i.e., `2019` to `2022`. These rates are based on recently auctioned bills with various maturities and collected by the Federal Reserve Bank of New York every business day. The Bank Discount rate is based on par value and discount amount, while the Coupon Equivalent is based on the purchase price and discount and can be used to compare yields. In this study, we use the `52 WEEKS COUPON EQUIVALENT` data as a surrogate for the effective market interest rate $\bar{r}$ available on each trading day.

### Implementation
We store the rate information in the `rate_data_dictionary` variable, which is a [Dictionary](https://docs.julialang.org/en/v1/base/collections/#Dictionaries); the keys of the dictionary are the years (from the `years` array, type `String`) while the values are a `DataFrame` holding the rate values. We load each year by iterating through the `years` array and passing the appropriate `year` parameter to the `loadratesfile(...)` function:

In [18]:
rate_data_dictionary = Dict{String,DataFrame}();
years = ["2019","2020","2021","2022","2023"];
for year ∈ years
    rate_data_dictionary[year] = loadratesfile(year=year);
end

### Select a year of interest rate data to analyze:
Let;s `2022` for the year to analyze, and save our selection in the variable `selected_year`. Then extract the corresponding rate values for the `selected_year` from the `rate_data_dictionary`. The rate data will be stored in the `dataset` variable (which is a `DataFrame`):

In [4]:
selected_year = "2022"
dataset = rate_data_dictionary[selected_year];

## Objective 2: Identify possible treasury bill trade signals
Whether to `hold` or `sell` a T-bill (or any other security) depends on your investment goals. Treasury securities, like T-bills, are generally considered safe, short-term investments. When you buy these securities, you receive their face value at the end of the term. However, you won’t have access to the invested funds until that time. Thus, if you’re concerned about accessing these funds before the term ends, such as for unexpected expenses, you could always sell the T-bill before maturity and potentially earn a capital gain or loss. However, when does this make sense?

You have two possible actions $a\in\left\{\text{sell},\text{hold}\right\}$ and you propose the following trade policy:
* You implement a `sell` action if the T-bill experiences a $\alpha$% gain relative to the purchase price
* Otherwise, you implement a `hold` action which earns effective market rate of interest $\bar{r}$.

### Prerequisite: Compute the price of a 52-week T-bill for each trading day of 2022
To calculate the cost of a 52-week T-bill for each trading day in 2022, start by finding out how many trading days are in `2022` using the `nrow(…)` function. Save this number in the `number_of_trading_days` variable. Then create a one-dimensional `models` array containing instances of `MyUSTreasuryBillModel`. Finally, using a `for` loop, construct `MyUSTreasuryBillModel` instances,  and compute the price, for a 52-week treasury bill using the effective market rate of interest $\bar{r}$ (found in the `dataset` variable) for each trading day. Store these instances in the `models` array.

__Reminder__: the `dataset` is organized newest to oldest.

In [5]:
number_of_trading_days = nrow(dataset)
models = Array{MyUSTreasuryBillModel,1}(undef, number_of_trading_days);
for i ∈ 1:number_of_trading_days
    
    r̄ = dataset[i,key]*(1/100)
    model = build(MyUSTreasuryBillModel, (
        par = Vₚ, T = T, rate=r̄)) |> compounding
    
    models[i] = model;
end

In [6]:
models

249-element Vector{MyUSTreasuryBillModel}:
 MyUSTreasuryBillModel(100.0, 0.0451, 1.0, 95.68462348100661)
 MyUSTreasuryBillModel(100.0, 0.044800000000000006, 1.0, 95.71209800918837)
 MyUSTreasuryBillModel(100.0, 0.0449, 1.0, 95.70293808019906)
 MyUSTreasuryBillModel(100.0, 0.0452, 1.0, 95.67546880979718)
 MyUSTreasuryBillModel(100.0, 0.0446, 1.0, 95.73042312847024)
 MyUSTreasuryBillModel(100.0, 0.044500000000000005, 1.0, 95.73958831977023)
 MyUSTreasuryBillModel(100.0, 0.0441, 1.0, 95.77626664112633)
 MyUSTreasuryBillModel(100.0, 0.0444, 1.0, 95.74875526618155)
 MyUSTreasuryBillModel(100.0, 0.0444, 1.0, 95.74875526618155)
 MyUSTreasuryBillModel(100.0, 0.0441, 1.0, 95.77626664112633)
 MyUSTreasuryBillModel(100.0, 0.0444, 1.0, 95.74875526618155)
 MyUSTreasuryBillModel(100.0, 0.0444, 1.0, 95.74875526618155)
 MyUSTreasuryBillModel(100.0, 0.0444, 1.0, 95.74875526618155)
 ⋮
 MyUSTreasuryBillModel(100.0, 0.0053, 1.0, 99.47279419078882)
 MyUSTreasuryBillModel(100.0, 0.0055000000000000005, 1.0, 

### Compute the return array going forward in time

In [12]:
reversed_models_array = reverse(models);
forward_return_array = Array{Float64,1}(undef, number_of_trading_days)
purchased_model = reversed_models_array[1];
for i ∈ 1:number_of_trading_days
    
    next_model = reversed_models_array[i];
    next_price = next_model.price
    purchased_price = purchased_model.price
    forward_return_array[i] = log(next_price/purchased_price)*100.0
end

In [13]:
forward_return_array

249-element Vector{Float64}:
  0.0
  0.01992428777263476
 -0.009960655419343637
 -0.03983667018122975
 -0.019920318790960026
 -0.04979335859488533
 -0.04979335859488533
 -0.05974905575079581
 -0.05974905575079581
 -0.08961020164532173
 -0.1592515515225884
 -0.13935897138934186
 -0.1691963578556769
  ⋮
 -3.955014312749268
 -3.955014312749268
 -3.926285559858369
 -3.955014312749268
 -3.955014312749268
 -3.926285559858369
 -3.9645887299139386
 -3.9741622304717317
 -4.031583994897632
 -4.0028772342584205
 -3.993306482468552
 -4.022015990297693

### Compute the return array going backward in time

In [16]:
reverse_return_array = Array{Float64,1}(undef, number_of_trading_days)
purchased_model = models[1]
for i ∈ 1:number_of_trading_days
    
    next_model = models[i];
    next_price = next_model.price
    purchased_price = purchased_model.price
    reverse_return_array[i] = log(next_price/purchased_price)*100.0
end

In [17]:
reverse_return_array

249-element Vector{Float64}:
  0.0
  0.028709507829137216
  0.019138756039270087
 -0.009568004599934869
  0.04785375982596495
  0.05742726038375007
  0.09573043043933191
  0.06700167754842812
  0.06700167754842812
  0.09573043043933191
  0.06700167754842812
  0.06700167754842812
  0.06700167754842812
  ⋮
  3.8826570189083567
  3.8627644387751072
  3.932405788652371
  3.9622669345468964
  3.9622669345468964
  3.9722226317028033
  3.9722226317028033
  4.002095671506733
  3.9821793201164706
  4.01205533487834
  4.041940278070325
  4.022015990297688